In [1]:
import scipy as sp
import scipy.integrate as spint8
import GWGen; from GWGen import WFGenerator
from GWGen.WFGenerator import *
from few.utils.utility import get_separatrix
from superrad import ultralight_boson;
import time
import cProfile, pstats
from pstats import SortKey

M = 1e6
mu = 1e1
procamass=1e-18 # eV
a = 0.6
p0 = 10.0
e0 = 0.2
iota0 = 0.0
Y0 = np.cos(iota0)
Phi_phi0 = 0.
Phi_theta0 =0.
Phi_r0 = 0.
qS = 0.2
phiS = 0.2
qK = 0.8
phiK = 0.8
dist = 1.0
mich = False
dt = 15.0
T = 2

In [2]:
diffeq = PN(M,mu)
pr = cProfile.Profile()
pr.enable()
diffeq(0,[6.,0.7,0,0])
pr.disable()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats('tottime')
ps.print_stats()

         151 function calls (147 primitive calls) in 0.013 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.010    0.010    0.010    0.010 {pyUtility.pyGetSeparatrix}
        1    0.003    0.003    0.013    0.013 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/few/utils/utility.py:363(get_separatrix)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        1    0.000    0.000    0.013    0.013 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/GWGen/WFGenerator/Kludge.py:101(__call__)
        1    0.000    0.000    0.000    0.000 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/GWGen/Utils/Kerr.py:64(<lambda>)
        4    0.000    0.000    0.000    0.000 /home/shaunf/.local/lib/python3.7/site-packages/IPython/core/compilerop.py:166(extra_flags)
        1    0.000    0.0

In [3]:
traj = PNTraj()
pr = cProfile.Profile()
pr.enable()
traj(M,mu,a,p0,e0,Y0)
pr.disable()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats('tottime')
ps.print_stats()

         194536 function calls (187444 primitive calls) in 0.319 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1716    0.095    0.000    0.104    0.000 {pyUtility.pyGetSeparatrix}
     1607    0.039    0.000    0.041    0.000 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/GWGen/Utils/Kerr.py:64(<lambda>)
     1607    0.030    0.000    0.031    0.000 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/GWGen/Utils/Kerr.py:69(<lambda>)
     1607    0.019    0.000    0.020    0.000 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/GWGen/Utils/Kerr.py:54(<lambda>)
     1607    0.019    0.000    0.020    0.000 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/GWGen/Utils/Kerr.py:59(<lambda>)
     1607    0.013    0.000    0.256    0.000 /mnt/Data_Volume/Documents/software/Anaconda

In [2]:
use_gpu = False
inspkw = {"max_init_len": int(1e3),"npoints":100}
sumkwargs = {"use_gpu": use_gpu, "pad_output": False}
emriwop = EMRIWaveform(inspiral_kwargs=inspkw.copy(),sum_kwargs=sumkwargs.copy(),use_gpu=use_gpu)
pr = cProfile.Profile()
pr.enable()
emriwop(M,mu,a,p0,e0,Y0,qS,phiS,qK,phiK,dist,T=T)
pr.disable()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats('tottime')
ps.print_stats()

/mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/few/summation/aakwave.py:228: UserWarning: Inclination trajectory includes values within 1e-6 of the poles. We shift these values automatically away from poles by 1e-6.
  "Inclination trajectory includes values within 1e-6 of the poles. We shift these values automatically away from poles by 1e-6."


         196152 function calls (188989 primitive calls) in 2.303 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.435    1.435    1.435    1.435 {pycpuAAK.pyWaveform}
        2    0.329    0.164    0.329    0.164 {method 'searchsorted' of 'numpy.ndarray' objects}
     1716    0.146    0.000    0.155    0.000 {pyUtility.pyGetSeparatrix}
        4    0.078    0.019    0.078    0.019 {method 'astype' of 'numpy.ndarray' objects}
        3    0.075    0.025    0.075    0.025 {built-in method numpy.arange}
     1607    0.039    0.000    0.041    0.000 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/GWGen/Utils/Kerr.py:64(<lambda>)
     1607    0.028    0.000    0.030    0.000 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/GWGen/Utils/Kerr.py:69(<lambda>)
        1    0.021    0.021    1.938    1.938 /mnt/Data_Volume/Documents/software/Anaco

In [3]:
ub = ultralight_boson.UltralightBoson(spin=1,model="relativistic");
emrip = EMRIWithProcaWaveform(inspiral_kwargs=inspkw.copy(), sum_kwargs=sumkwargs.copy(), use_gpu=False)
traj = PNTraj()

In [4]:
pr = cProfile.Profile()
pr.enable()
emrip(M,mu,procamass,a,p0,e0,Y0,T=T,UltralightBoson=ub,qS=qS,qK=qK,phiS=phiS,phiK=phiK,dist=dist,npoints=100)
pr.disable()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats('tottime')
ps.print_stats()

****************
Generating Trajectory for final black mass and spin 995730.0420975242 and 0.02980664747609163
****************
Integration halted before ending time. Reasons: Separatrix reached!
final mass 995730.0420975242
final spin 0.02980664747609163
****************
Generating Waveform
****************
         326195 function calls (319811 primitive calls) in 4.421 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     5564    2.173    0.000    2.173    0.000 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/scipy/interpolate/fitpack2.py:1054(integral)
        1    1.225    1.225    1.225    1.225 {pycpuAAK.pyWaveform}
     1512    0.111    0.000    0.121    0.000 {pyUtility.pyGetSeparatrix}
     4173    0.075    0.000    0.075    0.000 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/superrad/matched_waveform.py:104(mass_cloud)
     5657    0.072